In [1]:

import pickle

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [2]:

from sklearn.pipeline import make_pipeline

In [3]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("green-taxi-duration")

2024/06/24 01:42:56 INFO mlflow.tracking.fluent: Experiment with name 'green-taxi-duration' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1719173576872, experiment_id='1', last_update_time=1719173576872, lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [4]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [5]:
# df_train = read_dataframe('data/green_tripdata_2021-01.parquet')
# df_val = read_dataframe('data/green_tripdata_2021-02.parquet')

df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')


target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [14]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model")

    # When we are using the pipeline, we do not need tp specifically download the dictvectorizer as binary.

    # with open('dict_vectorizer.bin', 'wb') as f_out:
    #     pickle.dump(dict_train,f_out)

    # mlflow.log_artifact('dict_vectorizer.bin')

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.7558229919200725


/home/ubuntu/.local/share/virtualenvs/web-service-mlflow-ZkaECBB5/lib/python3.9/site-packages/_distutils_hack/__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/home/ubuntu/.local/share/virtualenvs/web-service-mlflow-ZkaECBB5/lib/python3.9/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [7]:
from mlflow.tracking import MlflowClient

In [10]:
MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'
RUN_ID = '58062e56f2ea48bd92dfd4f72dcdaba5' # Get the RUN ID from the mlflow URL

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [11]:
path = client.download_artifacts(run_id=RUN_ID, path='dict_vectorizer.bin')

In [12]:
with open(path, 'rb') as f_out:
    dv = pickle.load(f_out)

In [13]:
dv

[{'PU_DO': '43_151', 'trip_distance': 1.01},
 {'PU_DO': '166_239', 'trip_distance': 2.53},
 {'PU_DO': '41_42', 'trip_distance': 1.12},
 {'PU_DO': '168_75', 'trip_distance': 1.99},
 {'PU_DO': '75_75', 'trip_distance': 0.45},
 {'PU_DO': '225_265', 'trip_distance': 12.19},
 {'PU_DO': '244_244', 'trip_distance': 3.39},
 {'PU_DO': '75_213', 'trip_distance': 6.69},
 {'PU_DO': '74_238', 'trip_distance': 2.34},
 {'PU_DO': '74_60', 'trip_distance': 5.48},
 {'PU_DO': '42_41', 'trip_distance': 0.9},
 {'PU_DO': '74_116', 'trip_distance': 2.08},
 {'PU_DO': '116_143', 'trip_distance': 4.64},
 {'PU_DO': '75_42', 'trip_distance': 1.68},
 {'PU_DO': '74_75', 'trip_distance': 0.68},
 {'PU_DO': '7_82', 'trip_distance': 2.7},
 {'PU_DO': '152_117', 'trip_distance': 29.07},
 {'PU_DO': '116_74', 'trip_distance': 2.78},
 {'PU_DO': '116_69', 'trip_distance': 2.25},
 {'PU_DO': '259_116', 'trip_distance': 0.0},
 {'PU_DO': '247_167', 'trip_distance': 1.03},
 {'PU_DO': '166_41', 'trip_distance': 0.65},
 {'PU_DO': '